# Battle of the Cities: Testing Foursquare API

In [11]:
import requests # library to handle requests
import pandas as pd 
import numpy as np 
import random 
import geopandas as gpd
import json

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values 

import folium

print('Folium installed')
print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Folium installed
Libraries imported.


In [242]:
sf_data = gpd.read_file('../data/sf-multifamily-merged.geojson')
sf_data.drop(['id'], axis=1, inplace=True)

sf_latlong = pd.read_csv("../data/sf-locations.csv",delimiter=",")

In [244]:
sf_latlong.head()

,Neighborhood,Latitude,Longitude
0,Anza Vista,37.7808,-122.4432
1,Alamo Square,37.7764,-122.4346
2,Sutro Heights,37.7782,-122.5083
3,Seacliff,37.7868,-122.4890
4,Lake Street,37.7856,-122.4794


## Foursquare API
### Credentials

In [ ]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

### Generate URL for get request

In [240]:
neighborhood_latitude = sf_latlong.loc[3, 'Latitude']
neighborhood_longitude = sf_latlong.loc[3, 'Longitude']
LIMIT = 100
radius = 300

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=IW1DNBRJOUHECGXR0ST002ZPDJYOBCIRTROZZ5YJZFHCIP3Q&client_secret=TWMIYINYFXOLI0IGNPXXLCIRHFWLUNJ4VP2TJZXSJGUP2ZBN&v=20180605&ll=37.7868,-122.48899999999999&radius=300&limit=100'

### Make request

In [241]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5dd4263678a484001b722d4f'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'San Francisco',
  'headerFullLocation': 'San Francisco',
  'headerLocationGranularity': 'city',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 37.7895000027, 'lng': -122.48558993488405},
   'sw': {'lat': 37.7840999973, 'lng': -122.49241006511593}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '408aff80f964a520c4f21ee3',
       'name': 'China Beach',
       'location': {'address': 'Sea Cliff Ave',
        'crossStreet': 'at El Camino Del Mar',
        'lat': 37.788089862931265,
        'lng': -122.49118566513062,
        'lab

## Maps
### Format latitude and logitude points to be plotted

In [235]:
locations = sf_latlong[['Latitude', 'Longitude']]
locationlist = locations.values.tolist()
len(locationlist)
locationlist[7]

[37.7777, -122.4927]

In [237]:
state_geo = f'../data/sf-multifamily-merged.geojson'
state_data = sf_data

m = folium.Map(location=[37.7749, -122.4194], zoom_start=12,control_scale = True)

choropleth = folium.Choropleth(
    geo_data=state_geo,
    name='choropleth',
    data=sf_data,
    columns=['Neighborhood', 'Zillow Median'],
    key_on='feature.properties.Neighborhood',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    nan_fill_color='grey',
    nan_fill_opacity=0.4,
    bins=4,
    highlight=True,
    legend_name='Median Rent'
    
).add_to(m)

folium.LayerControl().add_to(m)

choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['Neighborhood','Zillow Median'], aliases=['Neighborhood', 'Median Rent'])
)

for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=sf_latlong['Neighborhood'][point]).add_to(m)

m.add_child(folium.LatLngPopup())

m

In [272]:
nyc_data = gpd.read_file('../data/nyc-multi-merged.geojson')
nyc_data.drop(['id'], axis=1, inplace=True)

nyc_latlong = pd.read_csv("../data/nyc-locations.csv",delimiter=",")
nyc_latlong.head()

,Neighborhood,Latitude,Longitude
0,Marble Hill,40.8758,-73.9105
1,Inwood,40.8688,-73.9220
2,Washington Heights,40.8492,-73.9359
3,Hamilton Heights,40.8259,-73.9492
4,Manhattanville,40.8166,-73.9552


In [267]:
locations = nyc_latlong[['Latitude', 'Longitude']]
locationlist = locations.values.tolist()
len(locationlist)
locationlist[7]

[40.7957, -73.9389]

In [273]:
state_geo = f'../data/nyc-multi-merged.geojson'
state_data = nyc_data

m = folium.Map(location=[40.7831, -73.9712], zoom_start=12)
bins_multi = list(nyc_data['MedianRent'].quantile([0, 0.25, 0.5, 0.75, 1]))

multi_choropleth = folium.Choropleth(
    geo_data=state_geo,
    name='Multifamily',
    data=nyc_data,
    columns=['Name', 'MedianRent'],
    key_on='feature.properties.Name',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    nan_fill_color='grey',
    nan_fill_opacity=0.4,
    bins=bins_multi,
    highlight=True,
    overlay=True,
    legend_name='Median Rent'
    
).add_to(m)

# folium.LayerControl().add_to(m)

multi_choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['Name','MedianRent'], aliases=['Neighborhood', 'Median Rent'])
)

for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=nyc_latlong['Neighborhood'][point]).add_to(m)

m.add_child(folium.LatLngPopup())

m